In [17]:
import pandas as pd
import numpy as np
import kagglehub
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from copy import deepcopy
import json

In [18]:
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")
df = pd.read_csv(path + '/creditcard.csv')

In [19]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [21]:
X, y = df.drop(columns="Class"), df["Class"]

### Undersampling

In [22]:
class_counts = y.value_counts()
minority_class = class_counts.idxmin()
minority_count = class_counts.min()
majority_class = class_counts.idxmax()

desired_ratio = 2
majority_count = minority_count * desired_ratio
sampling_strategy = {
    minority_class: minority_count,
    majority_class: majority_count
    }

In [23]:
rus = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [25]:

print(f"Imbalance ratio in training set: {(y_train.count()-np.sum(y_train))/np.sum(y_train)}")
print(f"Class proportion in testing set: {(y_test.count()-np.sum(y_test))/np.sum(y_test)}")

Imbalance ratio in training set: 1.942643391521197
Class proportion in testing set: 2.2527472527472527


Creating new smaller training data set that has both classes:

In [26]:
rng = np.random.RandomState(42)
labeled_X = pd.DataFrame()
labeled_y = pd.Series()
labeled_y_sum = 0
while not (labeled_y_sum > 0 and len(labeled_y) != labeled_y_sum):
    random_index = rng.choice(y_train.index.tolist())
    labeled_y = pd.concat([labeled_y, y_train.loc[[random_index]]])
    y_train = y_train.drop(random_index)
    labeled_X = pd.concat([labeled_X, X_train.loc[[random_index]]])
    X_train = X_train.drop(random_index)
    labeled_y_sum = np.sum(labeled_y)


C:\Users\Konrad\AppData\Local\Temp\ipykernel_22216\3949074733.py:7: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  labeled_y = pd.concat([labeled_y, y_train.loc[[random_index]]])


In [27]:
def get_one_class_probability(binary_proba):
    single_values = []
    for proba in binary_proba:
        single_values.append(round(proba[0], 4))
    return single_values

In [28]:
labeled_X_active = deepcopy(labeled_X)
labeled_X_passive = deepcopy(labeled_X)
labeled_y_active = deepcopy(labeled_y)
labeled_y_passive = deepcopy(labeled_y)

unlabeled_X_active = deepcopy(X_train)
unlabeled_X_passive = deepcopy(X_train)
unlabeled_y_active = deepcopy(y_train)
unlabeled_y_passive = deepcopy(y_train)


Active Learning


In [ ]:
ITERS = len(X_train)
results = {
    "training set size": len(X_train),
    "count of minority class members": int(np.sum(y_train)),
    "true labels": [int(i) for i in y_test.values],
    "models": []}
active_model = SVC(kernel="poly", probability=True)
for i in range(ITERS):
    active_model.fit(labeled_X_active, labeled_y_active)
    
    test_proba = active_model.predict_proba(X_test)
    one_class_proba = get_one_class_probability(test_proba)
    model = {"training instances": len(labeled_y_active), 
             "predictions": one_class_proba,
             "minority_class_examples_used": sum(labeled_y_active)}
    results["models"].append(model)
    
    probabilities = active_model.predict_proba(unlabeled_X_active)
    uncertainty = np.abs(probabilities[:, 0] - probabilities[:, 1])
    least_confident_index = np.argmin(uncertainty)
    least_confident_sample = unlabeled_X_active.iloc[least_confident_index]
    df_index_number = least_confident_sample.name
    
    labeled_X_active = pd.concat([labeled_X_active, least_confident_sample.to_frame().T])
    labeled_y_active = pd.concat([labeled_y_active, unlabeled_y_active.loc[[df_index_number]]])
    unlabeled_X_active = unlabeled_X_active.drop(df_index_number)
    unlabeled_y_active = unlabeled_y_active.drop(df_index_number)
    print(f"{i}/{ITERS-1}")



0/1177
1/1177


2/1177
3/1177
4/1177
5/1177
6/1177
7/1177
8/1177
9/1177
10/1177


In [ ]:
with open(f"linears_svm_results/linear_active_learning_IR{desired_ratio}.json", "w", encoding="utf-8") as json_file:
    json.dump(results, json_file, ensure_ascii=False, indent=4, separators=(',', ': '))

In [ ]:
ITERS = len(X_train)
results = {
    "training set size": len(X_train),
    "count of minority class members": int(np.sum(y_train)),
    "true labels": [int(i) for i in y_test.values],
    "models": []}
active_model = SVC(kernel="poly", probability=True)
for i in range(ITERS):
    active_model.fit(labeled_X_passive, labeled_y_passive)
    
    test_proba = active_model.predict_proba(X_test)
    one_class_proba = get_one_class_probability(test_proba)
    model = {"training instances": len(labeled_y_passive), 
             "predictions": one_class_proba,
             "minority_class_examples_used": sum(labeled_y_passive)}
    results["models"].append(model)
    
    random_record = unlabeled_X_passive.sample(n=1)
    labeled_X_passive = pd.concat([labeled_X_passive, random_record])
    labeled_y_passive = pd.concat([labeled_y_passive, unlabeled_y_passive.loc[random_record.index]])
    
    unlabeled_X_passive = unlabeled_X_passive.drop(random_record.index)
    unlabeled_y_passive = unlabeled_y_passive.drop(random_record.index)
    
    print(f"{i}/{ITERS-1}")


0/1177
1/1177
2/1177
3/1177
4/1177
5/1177
6/1177
7/1177
8/1177
9/1177
10/1177
11/1177
12/1177
13/1177
14/1177
15/1177
16/1177
17/1177
18/1177
19/1177
20/1177
21/1177
22/1177
23/1177
24/1177
25/1177
26/1177
27/1177
28/1177
29/1177
30/1177
31/1177
32/1177
33/1177
34/1177
35/1177
36/1177
37/1177
38/1177
39/1177
40/1177
41/1177
42/1177
43/1177
44/1177
45/1177
46/1177
47/1177
48/1177
49/1177
50/1177
51/1177
52/1177
53/1177
54/1177
55/1177
56/1177
57/1177
58/1177
59/1177
60/1177
61/1177
62/1177
63/1177
64/1177
65/1177
66/1177
67/1177
68/1177
69/1177
70/1177
71/1177
72/1177
73/1177
74/1177
75/1177
76/1177
77/1177
78/1177
79/1177
80/1177
81/1177
82/1177
83/1177
84/1177
85/1177
86/1177
87/1177
88/1177
89/1177
90/1177
91/1177
92/1177
93/1177
94/1177
95/1177
96/1177
97/1177
98/1177
99/1177
100/1177
101/1177
102/1177
103/1177
104/1177
105/1177
106/1177
107/1177
108/1177
109/1177
110/1177
111/1177
112/1177
113/1177
114/1177
115/1177
116/1177
117/1177
118/1177
119/1177
120/1177
121/1177
122/1177
123

In [16]:
with open(f"linears_svm_results/linear_passive_learning_IR{desired_ratio}.json", "w", encoding="utf-8") as json_file:
    json.dump(results, json_file, ensure_ascii=False, indent=4, separators=(',', ': '))